`Chart` sadly does not support heatmaps which I need badly. So quick googling revealed following:

 - https://www.reddit.com/r/haskell/comments/3a21tx/color_mapheat_map_plotting_without_gnuplot/
 - https://github.com/bgamari/chart-histogram/commits/heatmap

In [ ]:
{-# LANGUAGE TemplateHaskell #-}
import Control.Lens
import Control.Monad

import Data.Default
import Data.Colour
import Data.Colour.RGBSpace
import Data.Colour.RGBSpace.HSL
import Data.Colour.SRGB

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Backend

--import Graphics.Rendering.Chart.Types
--import Graphics.Rendering.Chart.Plot.Types
import Graphics.Rendering.Chart.Axis
-- import Graphics.Rendering.Chart.ColorMap
import Data.Colour

import Debug.Trace

In [ ]:

-- grey :: PlotValue z => AlphaColour 
grey = opaque . uncurryRGB (rgbUsingSpace sRGBSpace) . hsl 0 0 . toValue

fst3 (a,_,_) = a
snd3 (_,a,_) = a
thd3 (_,_,a) = a 

data HeatMap z x y = HeatMap
  { _heat_map_color_map :: z -> AlphaColour Double
  , _heat_map_values             :: [((x,y),(x,y),z)]  
  }

instance Default (HeatMap z x y) where
  def =  HeatMap
    { _heat_map_color_map = grey
    , _heat_map_values    = []
    }
  
instance (PlotValue z) => ToPlot (HeatMap z) where
  toPlot p = Plot { _plot_render     = renderHeatMap p
                  , _plot_legend     = []
                  , _plot_all_points = ( (\((x1,_),(x2,_),_) -> [x1,x2]) =<< _heat_map_values p
                                       , (\((_,y1),(_,y2),_) -> [y1,y2]) =<< _heat_map_values p )
                  }
                  
renderHeatMap  :: (PlotValue z) =>
                  HeatMap z x y -> PointMapFn x y -> ChartBackend ()
renderHeatMap p pmap = forM_ (_heat_map_values p) $ \((x1,y1),(x2,y2),z) -> do
  let corner1 = pmap (LValue x1, LValue y1) 
      corner2 = pmap (LValue x2, LValue y2) 
      rect   = Rect corner1 corner2
      path   = rectPath rect  
  withFillStyle (FillStyleSolid $ _heat_map_color_map p z) $
      fillPath path
  where

makeLenses ''HeatMap

In [ ]:
hm :: HeatMap Double Double Double
hm = (defaultHeatMap :: HeatMap Double Double Double)
  { _heat_map_values = [((0.5,0.5), (0.6,0.6), 0.1)
                       ,((0.6,0.6), (0.7,0.7), 0.3)
                       ]
  }
  
toRenderable 
  $ layout_plots .~ [toPlot hm]
  $ (def :: Layout Double Double)